# <center>Challenge 0<center>

<p>Team Name:
<p>Student Names:

## Instructions
Use generic coding style unless hard-coded values are really necessary.<br>
Your code must be efficient and use self-explanatory naming.<br>
Use appropriate Python library methods for each task instead of using loops.<br>
Run your entire code and save. Then submit this <b>saved</b> copy.

## Imports

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# PART I

## Prepare Data

<b>A. Create a data set D with 100 data points with the following attributes (features) as such:</b><br>
seed = 0<br>
a1 is from a normal distribution N(mean=20,var=4)<br>
a2 is from a uniform distribution U(min=14,max=26)<br>

In [2]:
np.random.seed(0)
n = 100

# Create a1
a1_mean = 20
a1_var = 4
a1 = np.random.normal(loc=a1_mean, scale=np.sqrt(a1_var), size=n)

# Create a2
a2_min = 14
a2_max = 26
a2 = np.random.uniform(low=a2_min, high=a2_max, size=n)

# Create dataframe
D = pd.DataFrame({
    'a1': a1,
    'a2': a2
})

print(D)

           a1         a2
0   23.528105  19.086261
1   20.800314  21.276719
2   21.957476  14.230318
3   24.481786  17.618898
4   23.735116  21.922082
..        ...        ...
95  21.413146  23.972581
96  20.021000  21.547782
97  23.571741  24.471808
98  20.253824  17.282504
99  20.803979  23.576562

[100 rows x 2 columns]


x1 = 2*a1<br>
x2 = normal distribution N(mean=40,var=4)<br>
Pr(x1,x2,target) = {(x1, x2, 'P')}, data objects with class positive<br>
P = Rotate(Pr,45), rotate Pr by 45 degrees in counter-clockwise<br>
Nr = Shift(Pr,x1,20), shift in x1-dimension by 20 units towards right, data objects with class negative, where
Nr(x1,x2,target) = {(x1_shifted, x2, 'N')}<br>
N = Rotate(Nr,45), rotate Nr by 45 degrees in counter-clockwise<br>

In [5]:
def rotate(pts, degrees):
    """
    Rotate a given 2D point by a number of degrees with center of rotation at (0,0).
    pts = DataFrame with 2 attributes, representing x1 and x2
    degrees = Angle in the counter-clockwise direction.
    """
    radians = math.radians(degrees)
    R = []
    for x1, x2 in pts:
        rx1 = x1 * math.cos(radians) - x2 * math.sin(radians)
        rx2 = x1 * math.sin(radians) + x2 * math.cos(radians)
        R.append((rx1, rx2))
    #
    return R
#

def shift(pts, axis, shiftValue):
    """
    Shift a given 2D point by a set value on either axis.
    pts = DataFrame with 2 attributes, representing x1 and x2
    axis = Either x1 or x2
    shiftValue = Value to shift by on either axis
    """
    S = []
    # Shift on x1 axis
    if axis == 'x1':
        for x1, x2 in pts:
            sx1 = x1 + shiftValue
            S.append((sx1, x2))
        #
    elif axis == 'x2':
        for x1, x2 in pts:
            sx2 = x2 + shiftValue
            S.append((x1, sx2))
        #
    else:
        print('Invalid axis provided to function shift().')
        sys.exit()
    #
    return S
#

# Create x1
D['x1'] = 2 * D['a1']

# Create x2
x2_mean = 40
x2_var = 4
D['x2'] = np.random.normal(loc=x2_mean, scale=np.sqrt(x2_var), size=len(D))

# Create Pr
Pr = D[['x1', 'x2']].copy()
Pr['target'] = 'P'

# Create P
Pr_x = Pr[['x1', 'x2']].values
Pr_x_rotate = rotate(Pr_x, 45)
P = pd.DataFrame(Pr_x_rotate, columns=['x1', 'x2'])

# Create Nr
Pr_x_shift = shift(Pr_x, 'x1', 20)
Nr = pd.DataFrame(Pr_x_shift, columns=['x1', 'x2'])
Nr['target'] = 'N'

# Create N
Nr_x = Nr[['x1', 'x2']].values
Nr_x_rotate = rotate(Nr_x, 45)
N = pd.DataFrame(Nr_x_rotate, columns=['x1', 'x2'])

print('\nDataFrame D =========================')
print(D)
print('\nDataFrame Pr =========================')
print(Pr)
print('\nDataFrame P =========================')
print(P)
print('\nDataFrame Nr =========================')
print(Nr)
print('\nDataFrame N =========================')
print(N)


DataFrame D =========================
           a1         a2         x1         x2
0   23.528105  19.086261  47.056209  40.098990
1   20.800314  21.276719  41.600629  40.987674
2   21.957476  14.230318  43.914952  41.286629
3   24.481786  17.618898  48.963573  36.858753
4   23.735116  21.922082  47.470232  39.586193
..        ...        ...        ...        ...
95  21.413146  23.972581  42.826293  40.502969
96  20.021000  21.547782  40.042000  35.967187
97  23.571741  24.471808  47.143482  38.921091
98  20.253824  17.282504  40.507648  39.448659
99  20.803979  23.576562  41.607957  38.580544

[100 rows x 4 columns]

DataFrame Pr =========================
           x1         x2 target
0   47.056209  40.098990      P
1   41.600629  40.987674      P
2   43.914952  41.286629      P
3   48.963573  36.858753      P
4   47.470232  39.586193      P
..        ...        ...    ...
95  42.826293  40.502969      P
96  40.042000  35.967187      P
97  47.143482  38.921091      P
98  40.507648

### Merge Data

Merge Pr and Nr into a single set PNr. 

In [6]:
PNr = pd.concat([Pr, Nr], ignore_index=True)

print(PNr)

            x1         x2 target
0    47.056209  40.098990      P
1    41.600629  40.987674      P
2    43.914952  41.286629      P
3    48.963573  36.858753      P
4    47.470232  39.586193      P
..         ...        ...    ...
195  62.826293  40.502969      N
196  60.042000  35.967187      N
197  67.143482  38.921091      N
198  60.507648  39.448659      N
199  61.607957  38.580544      N

[200 rows x 3 columns]


# PART II

## Visualize Data
The axis scale should be the same. The class-P and -N data points will in red-circle and blue-triangle, respectively.

Plot Pr and P in the same figure on a separate sub-plots of 1 by 2:

Plot Nr and N in the same figure on a separate sub-plots of 1 by 2:

Plot {PNr}:

# PART III

## Simple Classifier

Classifier SimpleClassifier:
Given data D(x1, x2),<br>
If x1 + x2 > median, then predicted class<br>
...    <b>N</b><br>
Else<br>
...    <b>P</b>

Obtain predicted class labels using SimpleClassifier:

Print output Y(x1,x2,True_Target, Predicted_Target)

Plot output Y. Use the same marker shape. Predicted Target colors: P in Red and N in blue.

Obtain the Confusion Matrix and print:

Claculate the Performance Scores of Accuracy, Recall (Sensitivity), Specificity, F1-score. Plot as bar-chart:

## <center> REFERENCES </center>
List resources (book, internet page, etc.) that you used to complete this challenge.